In [ ]:
import requests
import re
import json
import os
from bs4 import BeautifulSoup
from random import choice, uniform
from time import sleep, time, ctime
from tqdm import tqdm_notebook 

In [ ]:
def get_proxy_list():
    url = 'https://free-proxy-list.com'
    html = requests.get(url, headers={
        'User-Agent': 'Mozilla/5.0 (compatible; MSIE 9.0; Windows NT 6.1; Trident/5.0; FunWebProducts)'
    }).text
    
    soup = BeautifulSoup(html, "lxml")
    
    proxy_list = []
    for elem in soup.find_all('a', attrs={'href': re.compile("/proxyserver/.+")}):
        proxy_list.append(elem['alt'])
    return proxy_list

In [ ]:
useragents = open('useragents.txt', 'r', encoding='utf-8').read().split('\n')

In [ ]:
url = 'http://sitespy.ru/my-ip'

def get_ip(html):
    soup = BeautifulSoup(html, "lxml")
    ip = soup.find('span', class_='ip').text.strip()
    ua = soup.find('span', class_='ip').find_next_sibling('span').text.strip()
    return ip, ua

In [ ]:
def get_urls(soup, urls_list):
    for elem in soup.find_all('a', attrs={'class': 'item-description-title-link'}):
        if elem['href'] not in urls_list:
            urls_list.add(elem['href'])
    return urls_list

In [ ]:
def get_soup(url, proxy, useragents):
    proxies = {'https': 'https://' + choice(proxy)}
    useragent = {'User-Agent': choice(useragents)}
    
    try:
        html = requests.get('https://www.avito.ru' + url, headers=useragent, proxies=proxies).text
    except:
        print(url)
        return
    
    soup = BeautifulSoup(html, "lxml")
    return url, soup

In [ ]:
def collect_urls(urls_list, proxy, useragents, start):
    url = 'https://www.avito.ru/rossiya/komnaty?p='    
    while len(urls_list) <= 10000:
        
        for i in tqdm_notebook(range(start, 1000)):
            sleep(uniform(2, 5))
            soup = get_soup('https://www.avito.ru' + url + str(i), proxy, useragents)[1]
            if not soup:
                continue
            urls_list = get_urls(soup, urls_list)
    
    return urls_list

In [ ]:
proxy = get_proxy_list()
urls_list = collect_urls(urls_list, proxy, useragents, 0)

In [ ]:
with open('urls.json', 'w', encoding='utf-8') as fw:
    json.dump(list(urls_list), fw, ensure_ascii=False)

In [ ]:
def soup_find(soup, url, tag, class_):
    try:
        res = re.sub('  +', ' ', soup.find(tag, class_=class_).text.strip().replace('\n\n', '\n')) + '\n--\n'
        return res
    except:
        print('soup_find {} fails for {}'.format(class_, url))
        return ''

In [ ]:
def save_page_data(url, soup):
    title = soup_find(soup, url, 'span', 'title-info-title-text')
    time = 'Дата размещения: ' + soup_find(soup, url, 'div', 'title-info-metadata-item')
    price = 'Цена: ' + soup_find(soup, url, 'span', 'price-value-string js-price-value-string')
    seller = 'Продавец: ' + soup_find(soup, url, 'div', 'seller-info js-seller-info')
    info = 'Характеристики: ' + soup_find(soup, url, 'div', 'item-view-block')
    address = soup_find(soup, url, 'div', 'item-map-location').replace('Скрыть карту', '')
    text = 'Описание: ' + soup_find(soup, url, 'div', 'item-description')
    
    with open(os.getcwd() + '/corpus/' + url.replace('/', '--') + '.txt', 'w', encoding='utf-8') as fw:
        data = re.sub('\n +', '\n', title + time + price + seller + info + address + text)
        cleaned =  data.replace('\n\n', '\n')
        fw.write(cleaned)

In [ ]:
def get_urls_visited():
    urls_visited = set()
    
    for elem in os.listdir(os.getcwd() + '/corpus'):
        urls_visited.add(elem.replace('--', '/').replace('.txt', ''))
    
    return urls_visited

In [ ]:
def get_ads(urls, useragents, proxy): 
    urls_visited = get_urls_visited()
    
    for url in tqdm_notebook(urls): 
        if url in urls_visited:
            continue
        
        sleep(uniform(2, 5))
        
        url_soup = get_soup(url, useragents, proxy)
        if url_soup:
            save_page_data(url_soup[0], url_soup[1])
            urls_visited.add(url)

In [ ]:
with open('urls.json', 'r', encoding='utf-8') as f:
    urls = json.load(f)

In [ ]:
proxy = get_proxy_list()
get_ads(urls, proxy, useragents)

In [ ]:
len(os.listdir(os.getcwd() + '/corpus/'))